# Car angle predictor

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, Input, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import sys
sys.path.append('../../utils')
import config_handling as conf
from database import Database
from file_io import path_handler

In [2]:
config = conf.read_config('../../config/automotive.conf.ini')
config.read('config.ini')
connection_type = config['settings']['connection']
connection_type
user = config[connection_type]['user']
pw = config[connection_type]['pw']
host = config[connection_type]['host']
db = config[connection_type]['db']
port = config[connection_type].getint('port')
db = Database(host,
              port,
              user,
              pw,
              db
              )
db.connect()
#image directory: 
basedir = config['settings']['image_directory']


Connection established


In [3]:
get_tags_query = """
    SELECT * FROM angletags 
    JOIN images ON images.id = angletags.image_id
    WHERE angletags.manual_annotation = 1
"""
r = db.execute_query(get_tags_query, [])

In [ ]:
data = pd.DataFrame(r)
data.sample(5)
del(r)

In [18]:
data.sample(4)

,image_id,angle,manual_annotation,pk,id,listing_id,processed,use_image,certainty_of_outside_yolo,yolobox_top_left_x,yolobox_top_left_y,yolobox_bottom_right_x,yolobox_bottom_right_y,area,confidence,image_path,abs_path
2481,1305352,6,1,2483,1305352,83867,1,1,None,100.0,67.0,593.0,498.0,0.552311,0.757042,kia\Carens\868c58f0-fab0-4e78-9ab0-d126d21f9d1...,Z:\kia\Carens\868c58f0-fab0-4e78-9ab0-d126d21f...
613,828137,1,1,614,828137,52953,1,0,None,NaN,NaN,NaN,NaN,NaN,NaN,mercedes-benz\EQ-Klasse (alle)\d2e79130-d705-4...,Z:\mercedes-benz\EQ-Klasse (alle)\d2e79130-d70...
342,577402,1,1,343,577402,36787,1,0,None,NaN,NaN,NaN,NaN,NaN,NaN,volkswagen\Beetle\4e0a8618-2fdb-4129-963f-6e00...,Z:\volkswagen\Beetle\4e0a8618-2fdb-4129-963f-6...
883,1061130,1,1,884,1061130,68496,1,0,None,NaN,NaN,NaN,NaN,NaN,NaN,opel\Astra\3ed28570-a20d-45c6-a2b8-24f5969b34d...,Z:\opel\Astra\3ed28570-a20d-45c6-a2b8-24f5969b...


In [19]:
## PROJECT CONSTANTS CONFIGURATION OPTIONS
# Configuration
USE_BBOX = False  # Toggle this to use bounding box coordinates
IMG_SIZE = (128, 128)  # Image dimensions
BATCH_SIZE = 32
#from DB
CLASSES = data.angle.unique()
NUM_CLASSES = len(CLASSES)
CLASS_MAPPING = {cls: idx for idx, cls in enumerate(CLASSES)}


In [20]:
# Map angle to integers
data['angle'] = data['angle'].map(CLASS_MAPPING)


In [21]:

# Preprocessing Function
def preprocess_image(image_path, bbox=None):
    image = tf.keras.utils.load_img(image_path, target_size=IMG_SIZE)
    image = tf.keras.utils.img_to_array(image) / 255.0
    if USE_BBOX and bbox is not None:
        #crop image if bbox constant is true.
        bbox = np.array(bbox) / np.array([IMG_SIZE[1], IMG_SIZE[0], IMG_SIZE[1], IMG_SIZE[0]])
        return image, bbox
    return image, None

# Data Generator
def data_generator(df, batch_size=BATCH_SIZE):
    images = []
    labels = []
    while True:
        for _, row in df.iterrows():
            image, bbox = preprocess_image(
                row['abs_path'],
                [row['yolobox_top_left_x'], row['yolobox_top_left_y'], row['yolobox_bottom_right_x'], row['yolobox_bottom_right_y']]
            )
            label = row['angle']
            images.append(image)
            labels.append(tf.keras.utils.to_categorical(label, num_classes=NUM_CLASSES))

            if len(images) == batch_size:
                yield np.array(images), np.array(labels)
                images, labels = [], []



In [22]:
# combine config readable basedir with the relative path in the dataframe: 
data['abs_path'] = data.apply(lambda row: path_handler(basedir, row.image_path), axis=1)

## data splitting:

In [23]:
# Split Data
train_df = data.sample(frac=0.8, random_state=42)
val_df = data.drop(train_df.index)

# Create Datasets
train_gen = data_generator(train_df)
val_gen = data_generator(val_df)


In [24]:
# Build Model
if USE_BBOX:
    image_input = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3), name="image")
    bbox_input = Input(shape=(4,), name="bbox")
    
    # Image branch
    x = Conv2D(32, (3, 3), activation='relu')(image_input)
    x = Flatten()(x)

    # Combine image and bbox inputs
    combined = Concatenate()([x, bbox_input])
    
    # Fully connected layers
    x = Dense(128, activation='relu')(combined)
    x = Dropout(0.5)(x)
    output = Dense(NUM_CLASSES, activation='softmax')(x)

    model = tf.keras.Model(inputs=[image_input, bbox_input], outputs=output)
else:
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(NUM_CLASSES, activation='softmax')
    ])


c:\Users\Frede\Documents\GitHub\automotive_project\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:

# Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [26]:


# Train Model
history = model.fit(
    train_gen,
    steps_per_epoch = len(train_df) // BATCH_SIZE,
    validation_data=val_gen,
    validation_steps = len(val_df) // BATCH_SIZE,
    epochs=10
)


Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 19s 279ms/step - accuracy: 0.4407 - loss: 18.1623 - val_accuracy: 0.5918 - val_loss: 1.5702
Epoch 2/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 19s 298ms/step - accuracy: 0.5084 - loss: 1.5877 - val_accuracy: 0.6035 - val_loss: 1.3248
Epoch 3/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 18s 276ms/step - accuracy: 0.5791 - loss: 1.3006 - val_accuracy: 0.6133 - val_loss: 1.1726
Epoch 4/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 18s 274ms/step - accuracy: 0.6169 - loss: 1.1124 - val_accuracy: 0.5957 - val_loss: 1.1120
Epoch 5/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 18s 279ms/step - accuracy: 0.6470 - loss: 0.9384 - val_accuracy: 0.6445 - val_loss: 1.0579
Epoch 6/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 18s 281ms/step - accuracy: 0.6611 - loss: 0.8577 - val_accuracy: 0.6309 - val_loss: 1.1207
Epoch 7/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 18s 281ms/step - accuracy: 0.6789 - loss: 0.7586 - val_accuracy: 0.6289 - val_loss: 1.1242
Epoch 8/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 18s 283ms/step - accuracy: 0.7098 - loss: 0.7201 - val_acc

In [27]:

# Evaluation and Confusion Matrix
def evaluate_model(df, generator):
    true_labels = []
    pred_labels = []
    for _, row in df.iterrows():
        image, bbox = preprocess_image(row['abs_path'], [row['yolobox_top_left_x'], row['yolobox_top_left_y'], row['yolobox_bottom_right_x'], row['yolobox_bottom_right_y']])
        label = row['angle']
        true_labels.append(label)
        print(image.shape)
        
        if USE_BBOX and bbox is not None:
            prediction = model.predict([[image], [bbox]], verbose=0)
        else:
            prediction = model.predict([image], verbose=0)
        pred_labels.append(np.argmax(prediction))

    cm = confusion_matrix(true_labels, pred_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=CLASSES)
    disp.plot(cmap=plt.cm.Blues)
    plt.show()

    print(classification_report(true_labels, pred_labels, target_names=CLASSES))

    # Custom Proximity Evaluation
    proximity_score = 0
    for true, pred in zip(true_labels, pred_labels):
        true_idx = CLASSES.index(CLASSES[true])
        pred_idx = CLASSES.index(CLASSES[pred])
        proximity_score += 1 / (1 + abs(true_idx - pred_idx))  # Higher score for closer predictions
    proximity_score /= len(true_labels)

    print(f"Custom Proximity Score: {proximity_score:.4f}")

# Evaluate
print("Validation Set Results:")
evaluate_model(val_df, val_gen)


Validation Set Results:
(128, 128, 3)


c:\Users\Frede\Documents\GitHub\automotive_project\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_6
Received: inputs=('Tensor(shape=(32, 128, 3))',)
  warnings.warn(msg)


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(32, 128, 3), dtype=float32). Expected shape (None, 128, 128, 3), but input has incompatible shape (32, 128, 3)[0m

Arguments received by Sequential.call():
  • inputs=('tf.Tensor(shape=(32, 128, 3), dtype=float32)',)
  • training=False
  • mask=('None',)

In [17]:

# Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train Model
history = model.fit(
    train_gen,
    steps_per_epoch=len(train_df) // BATCH_SIZE,
    validation_data=val_gen,
    validation_steps=len(val_df) // BATCH_SIZE,
    epochs=10
)

# Evaluation and Confusion Matrix
def evaluate_model(df, generator):
    true_labels = []
    pred_labels = []
    for _, row in df.iterrows():
        image, bbox = preprocess_image(row['abs_path'], [row['yolobox_top_left_x'], row['yolobox_top_left_y'], row['yolobox_bottom_right_x'], row['yolobox_bottom_right_y']])
        label = row['angle']
        true_labels.append(label)
        
        if USE_BBOX and bbox is not None:
            prediction = model.predict([[image], [bbox]], verbose=0)
        else:
            prediction = model.predict([image], verbose=0)
        pred_labels.append(np.argmax(prediction))

    cm = confusion_matrix(true_labels, pred_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=CLASSES)
    disp.plot(cmap=plt.cm.Blues)
    plt.show()

    print(classification_report(true_labels, pred_labels, target_names=CLASSES))

    # Custom Proximity Evaluation
    proximity_score = 0
    for true, pred in zip(true_labels, pred_labels):
        true_idx = CLASSES.index(CLASSES[true])
        #TODOhereyouneedtomakeacircularscoringthingy;-1iscloseto0alsofixwhymyspacebarisnotworking!
        pred_idx = CLASSES.index(CLASSES[pred])
        proximity_score += 1 / (1 + abs(true_idx - pred_idx))  # Higher score for closer predictions
    proximity_score /= len(true_labels)

    print(f"Custom Proximity Score: {proximity_score:.4f}")

# Evaluate
print("Validation Set Results:")
evaluate_model(val_df, val_gen)


Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 39s 597ms/step - accuracy: 0.9914 - loss: 0.0259 - val_accuracy: 0.7480 - val_loss: 1.6192
Epoch 2/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 38s 590ms/step - accuracy: 0.9927 - loss: 0.0341 - val_accuracy: 0.7363 - val_loss: 1.7864
Epoch 3/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 39s 603ms/step - accuracy: 0.9935 - loss: 0.0282 - val_accuracy: 0.7168 - val_loss: 1.8866
Epoch 4/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 40s 624ms/step - accuracy: 0.9968 - loss: 0.0163 - val_accuracy: 0.7070 - val_loss: 2.1218
Epoch 5/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 42s 650ms/step - accuracy: 0.9979 - loss: 0.0098 - val_accuracy: 0.7012 - val_loss: 2.3396
Epoch 6/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 39s 617ms/step - accuracy: 0.9970 - loss: 0.0176 - val_accuracy: 0.7266 - val_loss: 2.1610
Epoch 7/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 38s 600ms/step - accuracy: 0.9937 - loss: 0.0285 - val_accuracy: 0.6895 - val_loss: 2.5171
Epoch 8/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 38s 602ms/step - accuracy: 0.9939 - loss: 0.0222 - val_accu

ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(32, 128, 3), dtype=float32). Expected shape (None, 128, 128, 3), but input has incompatible shape (32, 128, 3)[0m

Arguments received by Sequential.call():
  • inputs=('tf.Tensor(shape=(32, 128, 3), dtype=float32)',)
  • training=False
  • mask=('None',)